# 0. 구글 드라이브 연동 및 필수 라이브러리 설치 및 로드

### 구글 드라이브 연동

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### 필수 라이브러리 설치

In [28]:
!pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 416 kB 28.0 MB/s 


In [40]:
!pip install git+https://github.com/ssut/py-hanspell.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-ia2oerso
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-ia2oerso
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=375bf78a8ad9d10218e482a1cf92e8b137cb753d4f3a4c35477fdb01245851c0
  Stored in directory: /tmp/pip-ephem-wheel-cache-4ourylrn/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


### 필수 라이브러리 로드

In [41]:
import pandas as pd
import re
from soynlp.normalizer import *
import numpy as np
from hanspell import spell_checker

### 폴더 경로 설정

In [4]:
path = '/content/gdrive/MyDrive/FINAL_SUBMIT/'

# 1. NSMC 데이터 로드

### 데이터 로드

In [5]:
nsmc0 = pd.read_csv(path+'nsmc_data.txt', sep='\t')

### 가져온 데이터 확인

In [8]:
nsmc0.head(2)

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1


### sample의 개수 확인

In [9]:
print('nsmc0 sample 수: {}개'.format(nsmc0.shape[0]))

nsmc0 sample 수: 200000개


# 2. 데이터 정제

### 결측값 확인 및 제거(nsmc0 -> nsmc1)

In [15]:
nsmc1 = nsmc0.copy()

In [16]:
nsmc1.isnull().sum()

# documnet에 8개의 null 값이 있다는 것을 확인했다.

id          0
document    8
label       0
dtype: int64

In [17]:
nsmc1 = nsmc1.dropna(how='any')

In [18]:
nsmc1.isnull().sum()

# document에 있는 null 값이 사라진 것을 볼 수 있다.
# 주의: 인덱스에 구멍이 뚫렸다.

id          0
document    0
label       0
dtype: int64

### 중복된 값 확인 및 제거(nsmc1 -> nsmc2)

In [19]:
nsmc2 = nsmc1.copy()

In [20]:
print('nsmc2 데이터 샘플의 수: {}'.format(nsmc2.shape[0]))
print('nsmc2 데이터의 document 종류의 수: {}'.format(nsmc2['document'].nunique()))

# 중복된 값이 있음을 알 수 있다.

nsmc2 데이터 샘플의 수: 199992
nsmc2 데이터의 document 종류의 수: 194543


In [21]:
nsmc2 = nsmc2.drop_duplicates(subset=['document'], ignore_index=True)

In [23]:
print('nsmc2 데이터 샘플의 수: {}'.format(nsmc2.shape[0]))
print('nsmc2 데이터의 document 종류의 수: {}'.format(nsmc2['document'].nunique()))

# 중복된 값이 지워진 것을 볼 수 있다.

nsmc2 데이터 샘플의 수: 194543
nsmc2 데이터의 document 종류의 수: 194543


### 노이즈 제거 및 정규화(nsmc2 -> nsmc3)

In [24]:
nsmc3 = nsmc2.copy()

In [26]:
nsmc3['document'] = nsmc3['document'].map(lambda sent: re.sub('ㅜ', 'ㅠ', sent))

# ㅜ와 ㅠ는 같은 의미이므로 통일한다.

In [27]:
nsmc3['document'] = nsmc3['document'].map(lambda sent: re.sub('[^ㅋㅎㅡㅉㅠ 가-힣]', '', sent))

In [30]:
nsmc3['document'] = nsmc3['document'].map(lambda sent: emoticon_normalize(sent, num_repeats=2))

# 3개이상 연속된 문자를 2개로 바꾼다.(예시, 'ㅋㅋㅋㅋㅋ' -> 'ㅋㅋ')

In [31]:
nsmc3['document'] = nsmc3['document'].map(lambda sent: re.sub('ㅋㅋㅋ','ㅋㅋ',sent))
nsmc3['document'] = nsmc3['document'].map(lambda sent: re.sub('ㅎㅎㅎ','ㅎㅎ',sent))
nsmc3['document'] = nsmc3['document'].map(lambda sent: re.sub('ㅠㅠㅠ','ㅠㅠ',sent))
nsmc3['document'] = nsmc3['document'].map(lambda sent: re.sub('ㅡㅡㅡ','ㅡㅡ',sent))
nsmc3['document'] = nsmc3['document'].map(lambda sent: re.sub('ㅉㅉㅉ','ㅉㅉ',sent))

# 알 수 없는 이유로 연속된 3개의 문자는 2개의 문자로 바뀌지 않아서 직접 바꿔준다.

### 결측값 확인 및 삭제(nsmc3 -> nsmc4)

In [32]:
nsmc4 = nsmc3.copy()

In [34]:
nsmc4['document'] = nsmc4['document'].map(lambda sent: re.sub('^ +', '', sent))
nsmc4['document'].replace('', np.nan, inplace=True)

# nsmc3의 결측값은 Nan이 아닌 공백 이외의 문자가 없는 string으로 되어있다.
# 그러므로 직접 np.nan으로 채워야 한다.

In [35]:
nsmc4.isnull().sum()

# 결측값이 생긴 것을 볼 수 있다.

id             0
document    1108
label          0
dtype: int64

In [36]:
nsmc4 = nsmc4.dropna(how='any')

In [37]:
nsmc4.isnull().sum()

# 결측값이 없어진 것을 볼 수 있다.
# 주의: 인덱스의 오름차순이 깨져있다.

id          0
document    0
label       0
dtype: int64

### 중복된 값 확인 및 삭제(nsmc4 -> nsmc5)

In [38]:
nsmc5 = nsmc4.copy()

In [39]:
print('nsmc5 데이터 샘플의 수: {}'.format(nsmc5.shape[0]))
print('nsmc5 데이터의 document 종류의 수: {}'.format(nsmc5['document'].nunique()))

# 중복된 값이 있음을 알 수 있다.

nsmc5 데이터 샘플의 수: 193435
nsmc5 데이터의 document 종류의 수: 190368


In [45]:
nsmc5 = nsmc5.drop_duplicates(subset=['document'], ignore_index=True)

In [46]:
print('nsmc5 데이터 샘플의 수: {}'.format(nsmc5.shape[0]))
print('nsmc5 데이터의 document 종류의 수: {}'.format(nsmc5['document'].nunique()))

# 중복된 값이 대부분 사라진 것을 볼 수 있다.

nsmc5 데이터 샘플의 수: 190368
nsmc5 데이터의 document 종류의 수: 190368


### 띄어쓰기 및 맞춤법 교정(nsmc5 -> nsmc6)

In [48]:
def spell_check(sent):
  return spell_checker.check(sent).checked

# 문장을 넣으주면 맞춤이 교정된 문장을 반환하는 함수이다.

In [49]:
# 이 과정은 시간이 오래걸리기 때문에 팀원 3명에서 분할하여 계산했다.
# 즉, 각자 계산한 결과들을 불러와서 합쳐준다.

In [50]:
nsmc6_1 = pd.read_excel(path+'done_CJ.xlsx', engine='openpyxl')
nsmc6_2 = pd.read_excel(path+'done_KM.xlsx', engine='openpyxl')
nsmc6_3 = pd.read_excel(path+'done_MK.xlsx', engine='openpyxl')

In [51]:
nsmc6 = pd.concat([nsmc6_1, nsmc6_2, nsmc6_3], ignore_index=True)

In [52]:
nsmc6 = nsmc6[['document', 'label']]

### 최종 중복값 및 결측값 지우기(nsmc6 -> nsmc7)

In [54]:
nsmc7 = nsmc6.copy()

In [55]:
nsmc7.isnull().sum()

# 결측 값이 없다.

document    0
label       0
dtype: int64

In [56]:
print('nsmc7 데이터 샘플의 수: {}'.format(nsmc7.shape[0]))
print('nsmc7 데이터의 document 종류의 수: {}'.format(nsmc7['document'].nunique()))

# 중복된 값이 존재한다.

nsmc7 데이터 샘플의 수: 190530
nsmc7 데이터의 document 종류의 수: 189049


In [57]:
nsmc7 = nsmc7.drop_duplicates(subset=['document'], ignore_index=True)

In [58]:
print('nsmc7 데이터 샘플의 수: {}'.format(nsmc7.shape[0]))
print('nsmc7 데이터의 document 종류의 수: {}'.format(nsmc7['document'].nunique()))

# 중복된 값이 지워진 것을 볼 수 있다.

nsmc7 데이터 샘플의 수: 189049
nsmc7 데이터의 document 종류의 수: 189049


# 3. 데이터 내보내기

In [59]:
nsmc7.to_excel(path + 'nsmc_data_preprocessed.xlsx', index=False)